In [1]:
import os
import spacy
from spacy.tokens.doc import Doc
import pandas as pd

In [2]:
class DatasetConfig():
    input_path: str
    output_path: str
    sheet_name: str
    usecols: list[str]
    cols_to_save: list[str]

    def __init__(self, input_path: str, output_path: str, sheet_name: str, usecols: list[str], cols_to_save: list[str]):
        self.input_path = input_path
        self.output_path = output_path
        self.sheet_name = sheet_name
        self.usecols = usecols
        self.cols_to_save = cols_to_save

In [3]:
fakespeak_config = DatasetConfig(
    # file_path="/content/drive/My Drive/fake_news_over_time/Fakespeak_ENG_modified.xlsx",
    input_path="./data/Fakespeak-ENG/Fakespeak-ENG modified.xlsx",
    output_path="./data/Fakespeak-ENG/Analysis_output/Fakespeak_lexical_density.xlsx",
    sheet_name="Working",
    usecols=['ID', 'combinedLabel', 'originalTextType', 'originalBodyText', 'originalDateYear'],
    cols_to_save=["ID", "combinedLabel", "originalTextType", "originalDateYear", "lexical_density"]
)

misinfotext_config = DatasetConfig(
    input_path="./data/MisInfoText/PolitiFact_original_modified.xlsx",
    output_path="./data/MisInfoText/Analysis_output/MisInfoText_lexical_density.xlsx",
    sheet_name="Working",
    usecols=None,
    cols_to_save=["factcheckURL", "originalURL", "originalTextType", "originalDateYear", "lexical_density"]
)

In [4]:
using_dataset = fakespeak_config

In [5]:
dataset_df = pd.read_excel(
    using_dataset.input_path, 
    sheet_name=using_dataset.sheet_name, 
    usecols=using_dataset.usecols
)

In [6]:
dataset_df.head()

,ID,combinedLabel,originalTextType,originalBodyText,originalDateYear
0,Politifact_FALSE_Social media_687276,False,Social media,Mexico is paying for the Wall through the new ...,2019
1,Politifact_FALSE_Social media_25111,False,Social media,"Chuck Schumer: ""why should American citizens b...",2019
2,Politifact_FALSE_Social media_735424,False,Social media,Billions of dollars are sent to the State of C...,2019
3,Politifact_FALSE_Social media_594307,False,Social media,If 50 Billion $$ were set aside to go towards ...,2019
4,Politifact_FALSE_Social media_839325,False,Social media,Huge@#CD 9 news. \n@ncsbe\n sent letter to eve...,2019


## Calculating lexical density using spacy

Lexical density is calculated by number of lexical words (nouns, verbs, adjectives, adverbs) divided by number of total words.

In [7]:
nlp = spacy.load('en_core_web_md')

c:\Users\Adam\AppData\Local\Python\pythoncore-3.11-64\Lib\site-packages\coreferee\manager.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [8]:
dataset_df["doc"] = list(nlp.pipe(dataset_df["originalBodyText"]))

In [9]:
lexical_pos = ["PROPN", "NOUN", "VERB", "ADJ", "ADV"]

def calculate_lexical_density(doc: Doc):
    num_lexical_items = len([
        token
        for token in doc
        if token.pos_ in lexical_pos
    ])

    return num_lexical_items / len(doc)

In [10]:
dataset_df["lexical_density"] = dataset_df["doc"].apply(calculate_lexical_density)
dataset_df

,ID,combinedLabel,originalTextType,originalBodyText,originalDateYear,doc,lexical_density
0,Politifact_FALSE_Social media_687276,False,Social media,Mexico is paying for the Wall through the new ...,2019,"(Mexico, is, paying, for, the, Wall, through, ...",0.482143
1,Politifact_FALSE_Social media_25111,False,Social media,"Chuck Schumer: ""why should American citizens b...",2019,"(Chuck, Schumer, :, "", why, should, American, ...",0.480000
2,Politifact_FALSE_Social media_735424,False,Social media,Billions of dollars are sent to the State of C...,2019,"(Billions, of, dollars, are, sent, to, the, St...",0.454545
3,Politifact_FALSE_Social media_594307,False,Social media,If 50 Billion $$ were set aside to go towards ...,2019,"(If, 50, Billion, $, $, were, set, aside, to, ...",0.368421
4,Politifact_FALSE_Social media_839325,False,Social media,Huge@#CD 9 news. \n@ncsbe\n sent letter to eve...,2019,"(Huge@#CD, 9, news, ., \n, @ncsbe, \n , sent, ...",0.560000
...,...,...,...,...,...,...,...
2956,Politifact_Pants on Fire_Social media_876628,Pants on Fire,Social media,A great lesson in Optics 101: The Monroe Doctr...,2023,"(A, great, lesson, in, Optics, 101, :, The, Mo...",0.443396
2957,Politifact_Pants on Fire_Social media_231170,Pants on Fire,Social media,“One of these Joe’s is not like the other… one...,2023,"(“, One, of, these, Joe, ’s, is, not, like, th...",0.321429
2958,Politifact_Pants on Fire_Social media_874359,Pants on Fire,Social media,Autopsies Prove that COVID-19 is a Disseminate...,2020,"(Autopsies, Prove, that, COVID-19, is, a, Diss...",0.456897
2959,Politifact_Pants on Fire_Social media_635418,Pants on Fire,Social media,She collapsed when she saw jfk jr. as she was ...,2021,"(She, collapsed, when, she, saw, jfk, jr, ., a...",0.395833


In [11]:
grouped_by_year = dataset_df.groupby(by="originalDateYear")
years = grouped_by_year.groups
years_dfs = [grouped_by_year.get_group(year) for year in years]
years_dfs[0].head()

,ID,combinedLabel,originalTextType,originalBodyText,originalDateYear,doc,lexical_density
0,Politifact_FALSE_Social media_687276,False,Social media,Mexico is paying for the Wall through the new ...,2019,"(Mexico, is, paying, for, the, Wall, through, ...",0.482143
1,Politifact_FALSE_Social media_25111,False,Social media,"Chuck Schumer: ""why should American citizens b...",2019,"(Chuck, Schumer, :, "", why, should, American, ...",0.480000
2,Politifact_FALSE_Social media_735424,False,Social media,Billions of dollars are sent to the State of C...,2019,"(Billions, of, dollars, are, sent, to, the, St...",0.454545
3,Politifact_FALSE_Social media_594307,False,Social media,If 50 Billion $$ were set aside to go towards ...,2019,"(If, 50, Billion, $, $, were, set, aside, to, ...",0.368421
4,Politifact_FALSE_Social media_839325,False,Social media,Huge@#CD 9 news. \n@ncsbe\n sent letter to eve...,2019,"(Huge@#CD, 9, news, ., \n, @ncsbe, \n , sent, ...",0.560000


In [12]:
summary_df = pd.DataFrame(
    [df["lexical_density"].describe() for df in years_dfs],
    index=pd.Index(data=years, name="year")
)
summary_df

,count,mean,std,min,25%,50%,75%,max
year,,,,,,,,
2019,283.0,0.474566,0.077969,0.280000,0.425543,0.472727,0.510658,0.833333
2020,773.0,0.479122,0.087781,0.230769,0.428571,0.471916,0.519731,0.909091
2021,706.0,0.490197,0.086770,0.240000,0.436907,0.477870,0.525252,1.000000
2022,482.0,0.494287,0.090014,0.161290,0.439268,0.488889,0.531250,0.888889
2023,514.0,0.510641,0.097491,0.222222,0.449137,0.497559,0.554825,1.000000
2024,203.0,0.505427,0.092564,0.333333,0.434993,0.491228,0.552778,0.857143


In [13]:
only_news_blog_and_social_media_df = dataset_df[(dataset_df["originalTextType"] == "News and blog") | (dataset_df["originalTextType"] == "Social media")]
only_news_blog_and_social_media_df

,ID,combinedLabel,originalTextType,originalBodyText,originalDateYear,doc,lexical_density
0,Politifact_FALSE_Social media_687276,False,Social media,Mexico is paying for the Wall through the new ...,2019,"(Mexico, is, paying, for, the, Wall, through, ...",0.482143
1,Politifact_FALSE_Social media_25111,False,Social media,"Chuck Schumer: ""why should American citizens b...",2019,"(Chuck, Schumer, :, "", why, should, American, ...",0.480000
2,Politifact_FALSE_Social media_735424,False,Social media,Billions of dollars are sent to the State of C...,2019,"(Billions, of, dollars, are, sent, to, the, St...",0.454545
3,Politifact_FALSE_Social media_594307,False,Social media,If 50 Billion $$ were set aside to go towards ...,2019,"(If, 50, Billion, $, $, were, set, aside, to, ...",0.368421
4,Politifact_FALSE_Social media_839325,False,Social media,Huge@#CD 9 news. \n@ncsbe\n sent letter to eve...,2019,"(Huge@#CD, 9, news, ., \n, @ncsbe, \n , sent, ...",0.560000
...,...,...,...,...,...,...,...
2956,Politifact_Pants on Fire_Social media_876628,Pants on Fire,Social media,A great lesson in Optics 101: The Monroe Doctr...,2023,"(A, great, lesson, in, Optics, 101, :, The, Mo...",0.443396
2957,Politifact_Pants on Fire_Social media_231170,Pants on Fire,Social media,“One of these Joe’s is not like the other… one...,2023,"(“, One, of, these, Joe, ’s, is, not, like, th...",0.321429
2958,Politifact_Pants on Fire_Social media_874359,Pants on Fire,Social media,Autopsies Prove that COVID-19 is a Disseminate...,2020,"(Autopsies, Prove, that, COVID-19, is, a, Diss...",0.456897
2959,Politifact_Pants on Fire_Social media_635418,Pants on Fire,Social media,She collapsed when she saw jfk jr. as she was ...,2021,"(She, collapsed, when, she, saw, jfk, jr, ., a...",0.395833


In [14]:
grouped_by_year_news_blog_social_media = only_news_blog_and_social_media_df.groupby(by="originalDateYear")
years_news_blog_social_media_dfs = [grouped_by_year_news_blog_social_media.get_group(year) for year in years]
years_news_blog_social_media_dfs[0].head()

,ID,combinedLabel,originalTextType,originalBodyText,originalDateYear,doc,lexical_density
0,Politifact_FALSE_Social media_687276,False,Social media,Mexico is paying for the Wall through the new ...,2019,"(Mexico, is, paying, for, the, Wall, through, ...",0.482143
1,Politifact_FALSE_Social media_25111,False,Social media,"Chuck Schumer: ""why should American citizens b...",2019,"(Chuck, Schumer, :, "", why, should, American, ...",0.480000
2,Politifact_FALSE_Social media_735424,False,Social media,Billions of dollars are sent to the State of C...,2019,"(Billions, of, dollars, are, sent, to, the, St...",0.454545
3,Politifact_FALSE_Social media_594307,False,Social media,If 50 Billion $$ were set aside to go towards ...,2019,"(If, 50, Billion, $, $, were, set, aside, to, ...",0.368421
4,Politifact_FALSE_Social media_839325,False,Social media,Huge@#CD 9 news. \n@ncsbe\n sent letter to eve...,2019,"(Huge@#CD, 9, news, ., \n, @ncsbe, \n , sent, ...",0.560000


In [15]:
summary_news_blog_social_media_df = pd.DataFrame(
    [df["lexical_density"].describe() for df in years_news_blog_social_media_dfs],
    index=pd.Index(data=years, name="year")
)
summary_news_blog_social_media_df

,count,mean,std,min,25%,50%,75%,max
year,,,,,,,,
2019,279.0,0.474473,0.078435,0.280000,0.425000,0.472727,0.510658,0.833333
2020,766.0,0.478887,0.088107,0.230769,0.426186,0.471698,0.519407,0.909091
2021,696.0,0.490061,0.087282,0.240000,0.436620,0.477160,0.525374,1.000000
2022,476.0,0.494246,0.090524,0.161290,0.437970,0.488630,0.531250,0.888889
2023,513.0,0.510657,0.097585,0.222222,0.449091,0.497389,0.555556,1.000000
2024,200.0,0.505450,0.093226,0.333333,0.434248,0.490538,0.555818,0.857143


## Writing dataframes to excel spreadsheet

In [ ]:
writer = pd.ExcelWriter(using_dataset.output_path, engine="xlsxwriter")

for df, year in zip(years_dfs, years):
    df.to_excel(writer, sheet_name=str(year), index=False, columns=using_dataset.cols_to_save)

summary_df.to_excel(writer, sheet_name="Summary")

writer.close()

In [16]:
output_path = using_dataset.output_path
output_path_split = output_path.split("/")
output_path_split.insert(len(output_path_split) - 1, "news_blog_and_social_media")
output_path_news_blog_social_media = "/".join(output_path_split)
output_path_news_blog_social_media

'./data/Fakespeak-ENG/Analysis_output/news_blog_and_social_media/Fakespeak_lexical_density.xlsx'

In [19]:
writer = pd.ExcelWriter(output_path_news_blog_social_media, engine="xlsxwriter")

for df, year in zip(years_news_blog_social_media_dfs, years):
    df.to_excel(writer, sheet_name=str(year), index=False, columns=using_dataset.cols_to_save)

summary_news_blog_social_media_df.to_excel(writer, sheet_name="Summary")

writer.close()